In [1]:
#Install Packages
%pip install faiss-cpu
%pip install sentence-transformers

  Using cached faiss_cpu-1.8.0-cp39-cp39-win_amd64.whl (14.5 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\Equity_news_research\venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
  Using cached pillow-10.3.0-cp39-cp39-win_amd64.whl (2.5 MB)
  Using cached scipy-1.13.0-cp39-cp39-win_amd64.whl (46.2 MB)
  Using cached transformers-4.40.1-py3-none-any.whl (9.0 MB)
  Using cached scikit_learn-1.4.2-cp39-cp39-win_amd64.whl (10.6 MB)
  Using cached torch-2.3.0-cp39-cp39-win_amd64.whl (159.7 MB)
  Using cached filelock-3.14.0-py3-none-any.whl (12 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
  Using cached safetensors-0.4.3-cp39-none-win_amd64.whl (287 kB)
  Using cached tokenizers-0.19.1-cp39-none-win_amd64.whl (2.2 MB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
  Using cached tbb-2021.12.0-py3-none-win_amd64.whl (286 kB)
  U

You should consider upgrading via the 'd:\Equity_news_research\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [5]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [6]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


#### Step 1 : Create source embeddings for the text column

In [7]:
from sentence_transformers import SentenceTransformer

d:\Equity_news_research\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

d:\Equity_news_research\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nishanth\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\Equity_news_research\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureW

In [9]:
vectors.shape
(8, 768)

(8, 768)

In [10]:
dim = vectors.shape[1]
dim

768

#### Step 2 : Build a FAISS Index for vectors

In [11]:
import faiss
index = faiss.IndexFlatL2(dim)

#### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [12]:
index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001DE5125CEA0> >

#### Step 4 : Encode search text using same encorder and normalize the output vector

In [13]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [14]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [15]:
# faiss.normalize_L2(svec)

#### Step 5: Search for similar vector in the FAISS index created(screenshot)

In [17]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [18]:
I

array([[3, 2]], dtype=int64)

In [19]:
df.loc[I[0]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [20]:
search_query

'I want to buy a polo t-shirt'

In [21]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


You can see that the two results from the dataframe are similar to a search_query